We have support for Twitter and Reddit but want to add support for all social media platforms.

In [ ]:
!pip install playwright jmespath nested-lookup "scrapfly-sdk[all]" httpx[http2] nest-asyncio

In [ ]:
import json
import httpx
from parsel import Selector

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.54 Safari/537.36",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
}


# retrieve embed HTML
with httpx.Client(http2=True, headers=HEADERS) as client:
    response = client.get(
        "https://syndication.twitter.com/srv/timeline-profile/screen-name/scrapfly_dev"
    )
    assert response.status_code == 200
sel = Selector(response.text)
# find data cache:
data = json.loads(sel.css("script#__NEXT_DATA__::text").get())
# parse tweet data from data cache JSON:
tweet_data = data["props"]["pageProps"]["timeline"]["entries"]
tweets = [tweet["content"]["tweet"] for tweet in tweet_data]
print(tweets)

[]


In [ ]:
import httpx

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.54 Safari/537.36",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
}


# retrieve embed HTML
with httpx.Client(http2=True, headers=HEADERS) as client:
    response = client.get(
        "https://cdn.syndication.twimg.com/tweet-result?id=1664267318053179398&lang=en",
    )
    assert response.status_code == 200
    data = response.json()
    print(data)

{}


In [ ]:
}|import nest_asyncio
nest_asyncio.apply()

SyntaxError: ignored

In [ ]:
from playwright.sync_api import sync_playwright
from nested_lookup import nested_lookup
import asyncio
from playwright.async_api import async_playwright

async def scrape_profile(url: str) -> dict:
    """
    Scrapes Twitter user profile page e.g.:
    https://twitter.com/scrapfly_dev
    returns user data and latest tweets
    """
    _xhr_calls = []

    def intercept_response(response):
        """capture all background requests and save them"""
        # we can extract details from background requests
        if response.request.resource_type == "xhr":
            _xhr_calls.append(response)
        return response

    async with async_playwright() as pw:
        browser = pw.chromium.launch()
        context = browser.new_context(viewport={"width": 1920, "height": 1080})
        page = context.new_page()
        # enable intercepting for this page

        page.on("response", intercept_response)
        page.goto(url)
        page.wait_for_selector("[data-testid='tweet']")

        user_calls = [f for f in _xhr_calls if "UserBy" in f.url]
        users = {}
        for xhr in user_calls:
            data = xhr.json()
            user_data = data["data"]["user"]["result"]
            users[user_data["legacy"]["screen_name"]] = user_data

        tweet_calls = [f for f in _xhr_calls if "UserTweets" in f.url]
        tweets = []
        for xhr in tweet_calls:
            data = xhr.json()
            xhr_tweets = nested_lookup("tweet_results", data)
            tweets.extend([tweet["result"] for tweet in xhr_tweets])
            users[user_data["legacy"]["screen_name"]] = user_data

    return {"users": users, "tweets": tweets}


if __name__ == "__main__":
    print(asyncio.run(scrape_profile("https://twitter.com/Scrapfly_dev")))

Try Scrapify

In [ ]:
import json
from typing import Dict

from nested_lookup import nested_lookup
from scrapfly import ScrapeConfig, ScrapflyClient

SCRAPFLY = ScrapflyClient(key=os.environ["SCRAPFLY_KEY"])
BASE_CONFIG = {
    # Twitter.com requires Anti Scraping Protection bypass feature.
    # for more: https://scrapfly.io/docs/scrape-api/anti-scraping-protection
    "asp": True,
    # Twitter.com is javascript-powered web application so it requires
    # headless browsers for scraping
    "render_js": True,
    "country": "CA",  # set prefered country here, for example Canada
}

async def scrape_tweet(url: str) -> Dict:
    """
    Scrape a single tweet page for Tweet thread e.g.:
    https://twitter.com/Scrapfly_dev/status/1667013143904567296
    Return parent tweet, reply tweets and recommended tweets
    """
    result = await scrapfly.async_scrape(ScrapeConfig(
        url,
        wait_for_selector="[data-testid='tweet']"
        **BASE_CONFIG
    ))
    # capture background requests and extract ones that request Tweet data
    _xhr_calls = result.scrape_result["browser_data"]["xhr_call"]
    tweet_call = [f for f in _xhr_calls if "TweetDetail" in f["url"]]
    tweets = []
    for xhr in tweet_call:
        if not xhr["response"]:
            continue
        data = json.loads(xhr["response"]["body"])
        # find tweet_results key recursive as that's where tweet data is located
        xhr_tweets = nested_lookup("tweet_results", data)
        tweets.extend([parse_tweet(tweet["result"]) for tweet in xhr_tweets])

    # Now that we have all tweets we can parse them into a thread
    # The first tweet is the parent, the rest are replies or suggested tweets
    parent = tweets.pop(0)
    replies = []
    other = []
    for tweet in tweets:
        if tweet["conversation_id"] == parent["conversation_id"]:
            replies.append(tweet)
        else:
            other.append(tweet)
    return {
        "tweet": parent,
        "replies": replies,
        "other": other,  # ads, recommended etc
    }

if __name__ == "__main__":
    import asyncio
    asyncio.run(scrape_tweet("https://twitter.com/Scrapfly_dev/status/1664267318053179398")


Loaded Reply:

{
  "tweet": {
    "__typename": "Tweet",
    "rest_id": "1664267318053179398",
    "core": {
      "user_results": {
        "result": {
          "__typename": "User",
          "id": "VXNlcjoxMzEwNjIzMDgxMzAwNDAyMTc4",
          "rest_id": "1310623081300402178",
          "affiliates_highlighted_label": {},
          "is_blue_verified": true,
          "profile_image_shape": "Circle",
          "legacy": {
            "created_at": "Mon Sep 28 16:51:22 +0000 2020",
            "default_profile": true,
            "default_profile_image": false,
            "description": "Web Scraping API - turn any website into a database!\n\nScrapFly allows you to quickly achieve your data goals without web scraping challenges and errors.",
            "entities": {
              "description": {
                "urls": []
              },
              "url": {
                "urls": [
                  {
                    "display_url": "scrapfly.io",
                    "expanded_url": "https://scrapfly.io",
                    "url": "https://t.co/1Is3k6KzyM",
                    "indices": [
                      0,
                      23
                    ]
                  }
                ]
              }
            },
            "fast_followers_count": 0,
            "favourites_count": 26,
            "followers_count": 163,
            "friends_count": 993,
            "has_custom_timelines": true,
            "is_translator": false,
            "listed_count": 2,
            "location": "Paris",
            "media_count": 11,
            "name": "Scrapfly",
            "normal_followers_count": 163,
            "pinned_tweet_ids_str": [],
            "possibly_sensitive": false,
            "profile_banner_url": "https://pbs.twimg.com/profile_banners/1310623081300402178/1601320645",
            "profile_image_url_https": "https://pbs.twimg.com/profile_images/1310658795715076098/XedZDwC7_normal.jpg",
            "profile_interstitial_type": "",
            "screen_name": "Scrapfly_dev",
            "statuses_count": 56,
            "translator_type": "none",
            "url": "https://t.co/1Is3k6KzyM",
            "verified": false,
            "withheld_in_countries": []
          }
        }
      }
    },
    "edit_control": {
      "edit_tweet_ids": [
        "1664267318053179398"
      ],
      "editable_until_msecs": "1685629023000",
      "is_edit_eligible": true,
      "edits_remaining": "5"
    },
    "is_translatable": false,
    "views": {
      "count": "43",
      "state": "EnabledWithCount"
    },
    "source": "<a href=\"https://zapier.com/\" rel=\"nofollow\">Zapier.com</a>",
    "legacy": {
      "bookmark_count": 0,
      "bookmarked": false,
      "created_at": "Thu Jun 01 13:47:03 +0000 2023",
      "conversation_id_str": "1664267318053179398",
      "display_text_range": [
        0,
        122
      ],
      "entities": {
        "media": [
          {
            "display_url": "pic.twitter.com/zLjDlxdKee",
            "expanded_url": "https://twitter.com/Scrapfly_dev/status/1664267318053179398/photo/1",
            "id_str": "1664267314160607232",
            "indices": [
              123,
              146
            ],
            "media_url_https": "https://pbs.twimg.com/media/FxiqTffWIAALf7O.png",
            "type": "photo",
            "url": "https://t.co/zLjDlxdKee",
            "features": {
              "large": {
                "faces": []
              },
              "medium": {
                "faces": []
              },
              "small": {
                "faces": []
              },
              "orig": {
                "faces": []
              }
            },
            "sizes": {
              "large": {
                "h": 416,
                "w": 796,
                "resize": "fit"
              },
              "medium": {
                "h": 416,
                "w": 796,
                "resize": "fit"
              },
              "small": {
                "h": 355,
                "w": 680,
                "resize": "fit"
              },
              "thumb": {
                "h": 150,
                "w": 150,
                "resize": "crop"
              }
            },
            "original_info": {
              "height": 416,
              "width": 796,
              "focus_rects": [
                {
                  "x": 27,
                  "y": 0,
                  "w": 743,
                  "h": 416
                },
                {
                  "x": 190,
                  "y": 0,
                  "w": 416,
                  "h": 416
                },
                {
                  "x": 216,
                  "y": 0,
                  "w": 365,
                  "h": 416
                },
                {
                  "x": 294,
                  "y": 0,
                  "w": 208,
                  "h": 416
                },
                {
                  "x": 0,
                  "y": 0,
                  "w": 796,
                  "h": 416
                }
              ]
            }
          }
        ],
        "user_mentions": [],
        "urls": [
          {
            "display_url": "scrapfly.io/blog/top-10-we\u2026",
            "expanded_url": "https://scrapfly.io/blog/top-10-web-scraping-libraries-in-python/",
            "url": "https://t.co/d2iFdAV2LJ",
            "indices": [
              99,
              122
            ]
          }
        ],
        "hashtags": [],
        "symbols": []
      },
      "extended_entities": {
        "media": [
          {
            "display_url": "pic.twitter.com/zLjDlxdKee",
            "expanded_url": "https://twitter.com/Scrapfly_dev/status/1664267318053179398/photo/1",
            "id_str": "1664267314160607232",
            "indices": [
              123,
              146
            ],
            "media_key": "3_1664267314160607232",
            "media_url_https": "https://pbs.twimg.com/media/FxiqTffWIAALf7O.png",
            "type": "photo",
            "url": "https://t.co/zLjDlxdKee",
            "ext_media_availability": {
              "status": "Available"
            },
            "features": {
              "large": {
                "faces": []
              },
              "medium": {
                "faces": []
              },
              "small": {
                "faces": []
              },
              "orig": {
                "faces": []
              }
            },
            "sizes": {
              "large": {
                "h": 416,
                "w": 796,
                "resize": "fit"
              },
              "medium": {
                "h": 416,
                "w": 796,
                "resize": "fit"
              },
              "small": {
                "h": 355,
                "w": 680,
                "resize": "fit"
              },
              "thumb": {
                "h": 150,
                "w": 150,
                "resize": "crop"
              }
            },
            "original_info": {
              "height": 416,
              "width": 796,
              "focus_rects": [
                {
                  "x": 27,
                  "y": 0,
                  "w": 743,
                  "h": 416
                },
                {
                  "x": 190,
                  "y": 0,
                  "w": 416,
                  "h": 416
                },
                {
                  "x": 216,
                  "y": 0,
                  "w": 365,
                  "h": 416
                },
                {
                  "x": 294,
                  "y": 0,
                  "w": 208,
                  "h": 416
                },
                {
                  "x": 0,
                  "y": 0,
                  "w": 796,
                  "h": 416
                }
              ]
            }
          }
        ]
      },
      "favorite_count": 0,
      "favorited": false,
      "full_text": "A new blog post has been published! \n\nTop 10 Web Scraping Packages for Python \ud83e\udd16\n\nCheckout it out \ud83d\udc47\nhttps://t.co/d2iFdAV2LJ https://t.co/zLjDlxdKee",
      "is_quote_status": false,
      "lang": "en",
      "possibly_sensitive": false,
      "possibly_sensitive_editable": true,
      "quote_count": 0,
      "reply_count": 0,
      "retweet_count": 0,
      "retweeted": false,
      "user_id_str": "1310623081300402178",
      "id_str": "1664267318053179398"
    },
    "quick_promote_eligibility": {
      "eligibility": "IneligibleUserUnauthorized"
    }
  },
  "replies": [],
  "other": []
}

Reddit

In [54]:
!pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 3.5 MB/s eta 0:00:00


In [ ]:
# Redacted
reddit = praw.Reddit(client_id='my_client_id',
                     client_secret='my_client_secret',
                     user_agent='my_user_agent')
hot_posts = reddit.subreddit('Google').hot(limit=10)

News API

In [41]:
!pip install newsapi-python

In [42]:
from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key='redacted')

# /v2/everything
all_articles = newsapi.get_everything(q='openai',
                                      from_param='2020-08-01',
                                      to='2023-08-08',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)


ImportError: ignored

In [47]:
import requests
#we create URL for samsung for date 30 May 2020 sort by popularity
url = ('http://newsapi.org/v2/everything?' #search everything
       'q=samsung&'
       #q - Keywords or phrases to search for keyword in the article title and body.
       #qInTitle - request to Keywords or phrases to search for keyword in the article title only.
       'from=2023-07-20&'#from date
       'to=2020-07-25&'#to date
       'domains=bbc.co.uk&'#domains that we want
       'language=en&'#choose your language
       'sortBy=popularity&'#if we want to sort by popularity,relevancy or publishedAt
       'pageSize=30&' #if we want the number of articles default is 20 and maximum is 100
       'apiKey=redacted') #this is compulsory

#create request to get news article from newsapi
response = requests.get(url)

print(response.json()['articles'])

[{'source': {'id': 'bbc-news', 'name': 'BBC News'}, 'author': None, 'title': 'Huawei Mate X2: Mastering the folding phone formula', 'description': 'Huawei has revealed the third iteration of its folding phone, taking inspiration from Samsung.', 'url': 'https://www.bbc.co.uk/news/av/technology-56945791', 'urlToImage': 'https://ichef.bbci.co.uk/images/ic/400xn/p09g8v75.jpg', 'publishedAt': '2021-05-02T23:55:16Z', 'content': 'Huawei has revealed the third iteration of its folding smartphone, abandoning its previous designs and taking some inspiration from Samsung.\r\nThis time, the folding display on the Mate X2 is protecte… [+238 chars]'}, {'source': {'id': 'bbc-news', 'name': 'BBC News'}, 'author': 'https://www.facebook.com/bbcnews', 'title': 'Global chip shortage: Samsung forecasts 53% jump in quarterly profit', 'description': 'Demand for electronic components has surged as people are shifting more of their lives online.', 'url': 'https://www.bbc.co.uk/news/business-57744982', 'urlToIm

In [52]:
import pandas as pd
df = pd.DataFrame(response.json()['articles'])

In [53]:
df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': 'bbc-news', 'name': 'BBC News'}",None,Huawei Mate X2: Mastering the folding phone fo...,Huawei has revealed the third iteration of its...,https://www.bbc.co.uk/news/av/technology-56945791,https://ichef.bbci.co.uk/images/ic/400xn/p09g8...,2021-05-02T23:55:16Z,Huawei has revealed the third iteration of its...
1,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,Global chip shortage: Samsung forecasts 53% ju...,Demand for electronic components has surged as...,https://www.bbc.co.uk/news/business-57744982,https://ichef.bbci.co.uk/news/1024/branded_new...,2021-07-07T04:47:41Z,Tech giant Samsung Electronics has said it exp...
2,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,Z Fold 3: Samsung aims to take folding phones ...,The electronics giant hopes lower prices and i...,https://www.bbc.co.uk/news/technology-58175048,https://ichef.bbci.co.uk/news/1024/branded_new...,2021-08-11T14:03:37Z,image captionSamsung has revealed the Z Fold 3...
3,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,Lee Jae-yong: Samsung heir released from priso...,Critics say the move raises fresh questions ab...,https://www.bbc.co.uk/news/world-asia-58196575,https://ichef.bbci.co.uk/news/1024/branded_new...,2021-08-13T03:17:42Z,image captionLee has served 18 months of a rev...
4,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,Nike halts footwear production due to Covid ou...,Nike could run out of its Vietnamese-made trai...,https://www.bbc.co.uk/news/business-57902093,https://ichef.bbci.co.uk/news/1024/branded_new...,2021-07-20T13:11:04Z,By Beth TimminsBBC News\r\nimage copyrightGett...
5,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,Bosphorus Cross-Continental Swimming Race: Hun...,The race saw hundreds of competitors swim 6.5k...,https://www.bbc.co.uk/news/world-europe-53884731,https://ichef.bbci.co.uk/news/1024/branded_new...,2020-08-23T22:34:43Z,Image copyrightEPA\r\nWhile many sporting even...
6,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,Park Geun-hye: South Korea's ex-president gran...,Park had previously received a 22-year-old pri...,https://www.bbc.co.uk/news/world-asia-59777757,https://ichef.bbci.co.uk/news/1024/branded_new...,2021-12-24T01:17:07Z,"Image source, Getty Images\r\nSouth Korean Pre..."
7,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,US-China chip war: Netherlands moves to restri...,"The measures will affect Dutch firm ASML, whic...",https://www.bbc.co.uk/news/business-64897794,https://ichef.bbci.co.uk/news/1024/branded_new...,2023-03-09T03:46:28Z,The Dutch government says before the summer it...
8,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,Official who drained dam to retrieve phone is ...,The man ordered millions of litres to be drain...,https://www.bbc.co.uk/news/world-asia-india-65...,https://ichef.bbci.co.uk/news/1024/branded_new...,2023-05-31T07:06:04Z,An Indian official who made headlines after he...
9,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,Japan's push into 'deep tech' innovation,Japanese start-up firms are developing next-ge...,https://www.bbc.co.uk/news/business-62719774,https://ichef.bbci.co.uk/news/1024/branded_new...,2022-09-25T23:06:29Z,"By Mariko OiBBC News, Kyoto\r\nImagine if you ..."
